In [ ]:
import os
from dotenv import load_dotenv

import base64
import hashlib
import hmac
import time
import uuid
from zeep import Client, Plugin
from zeep.transports import Transport
from lxml import etree
import requests

In [ ]:
load_dotenv()

In [ ]:
ACCOUNT_ID = os.getenv('account')
CONSUMER_KEY = os.getenv('consumer_key')
CONSUMER_SECRET = os.getenv('consumer_secret')
TOKEN_ID = os.getenv('token_key')
TOKEN_SECRET = os.getenv('token_secret')

In [ ]:
wsdl_url = "https://webservices.netsuite.com/wsdl/v2024_2_0/netsuite.wsdl"
webservices_url = f"https://{ACCOUNT_ID.lower().replace('_', '-')}.suitetalk.api.netsuite.com/services/NetSuitePort_2024_2"

In [ ]:
class TokenPassportPlugin(Plugin):
    def egress(self, envelope, http_headers, operation, binding_options):
        nonce = base64.b64encode(uuid.uuid4().bytes).decode("utf-8").rstrip("=")
        timestamp = str(int(time.time()))

        base_string = (
            ACCOUNT_ID + "&" +
            CONSUMER_KEY  + "&" +
            TOKEN_ID + "&" +
            nonce + "&" +
            timestamp
        )

        key = f"{CONSUMER_SECRET}&{TOKEN_SECRET}"
        signature = base64.b64encode(
            hmac.new(
                key.encode("utf-8"),
                base_string.encode("utf-8"),
                hashlib.sha256
            ).digest()
        ).decode()

        token_passport = etree.Element("{urn:messages_2024_2.platform.webservices.netsuite.com}tokenPassport")

        for tag, value in {
            "account": ACCOUNT_ID,
            "consumerKey": CONSUMER_KEY,
            "token": TOKEN_ID,
            "nonce": nonce,
            "timestamp": timestamp,
        }.items():
            el = etree.Element(f"{{urn:messages_2024_2.platform.webservices.netsuite.com}}{tag}")
            el.text = value
            token_passport.append(el)

        sig_el = etree.Element("{urn:messages_2024_2.platform.webservices.netsuite.com}signature")
        sig_el.set("algorithm", "HMAC-SHA256")
        sig_el.text = signature
        token_passport.append(sig_el)

        header = etree.Element("{http://schemas.xmlsoap.org/soap/envelope/}Header")
        header.append(token_passport)

        envelope.insert(0, header)
        return envelope, http_headers

In [ ]:
session = requests.Session()
client = Client(wsdl=wsdl_url, transport=Transport(session=session), plugins=[TokenPassportPlugin()])
client.service._binding_options["address"] = webservices_url

In [ ]:
response = client.service.search({"recordType": "customer"})
response